In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import io
import base64
import datetime
import dash_bootstrap_components as dbc
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

In [2]:
# Initialize the Dash app
#app = dash.Dash(__name__)  
#new theme (SOLAR) from dash bootstrap components
#app = dash.Dash(external_stylesheets=[dbc.themes.CYBORG]) 
#app = dash.Dash(external_stylesheets=[dbc.themes.COSMO])
#app = dash.Dash(external_stylesheets=[dbc.themes.SOLAR]) 
app = dash.Dash(external_stylesheets=[dbc.themes.LUMEN])

In [3]:
# Define the layout of the app
app.layout = html.Div(#children = 
[
    html.H1("Clevered Dashboard", className="text-center fw-bold text-decoration-underline"),
    
    html.Div(id='upload-container', className='centered-container', children=[
            dcc.Upload(
                id='upload-data',
                children=html.Div(['Drag and Drop or ', html.A('Select CSV File')]),
                style={
                    'width': '50%',
                    'height': '60px',
                    'lineHeight': '60px',
                    'borderWidth': '1px',
                    'borderStyle': 'dashed',
                    'borderRadius': '5px',
                    'textAlign': 'center',
                    'margin': '10px auto',  # Center horizontally
                },
                multiple=False,
            ),
        ]),
    html.Div([dcc.Dropdown(id='month-dropdown',options=[
                {'label': 'January', 'value': '01'},
                {'label': 'February', 'value': '02'},
                {'label': 'March', 'value': '03'},
                {'label': 'April', 'value': '04'},
                {'label': 'May', 'value': '05'},
                {'label': 'June', 'value': '06'},
                {'label': 'July', 'value': '07'},
                {'label': 'August', 'value': '08'},
                {'label': 'September', 'value': '09'},
                {'label': 'October', 'value': '10'},
                {'label': 'November', 'value': '11'},
                {'label': 'December', 'value': '12'},
            ],
            placeholder='Select a month...',
            style={'width': '90%', 'margin': '15px'},
            className="px-2 bg-white border rounded-pill"
        ),
        
        dcc.Dropdown(id='name-dropdown',placeholder='Select a person...',style={'width': '90%', 'margin': '15px'},
            className="px-2 bg-light border rounded-pill"
        ),
    ], style={'display': 'flex'}),  # Set display style to 'flex' for the div to align items horizontally instead of the default vertical alignment
    
    html.Div(id = 'search-container', className = 'centered-container', style = {'margin': '10px auto'}, 
             children = [dcc.Input(id='manual-search-input', type='text', placeholder='Enter a name...'),
                         html.Button('Search', id='manual-search-button', n_clicks=0, className = "text-center")]), #does not have a placeholder as it is not a dcc component but an html component 

    html.Div(id='output-columns'),
    
    html.Div(id='output-person-info'),
    
    html.Div([
        html.H2("Current Month Analysis", className="text-center fw-bold"),# fst-italic text-decoration-line-through"),

        html.Div([html.P('Birthdays in this month', className = "fst-italic fw-bold", style={'width': '50%', 'margin': 'auto', 'text-align': 'center', 'font-size': '20px'}),
                  html.P('Anniversaries in this month', className = "fst-italic fw-bold", style={'width': '50%', 'margin': 'auto', 'text-align': 'center', 'font-size': '20px'})], 
                  style={'display': 'flex', 'justify-content': 'center'}),
        
        html.Div(id='current-month-info', className = 'text-dark')   # This div will hold the analysis information
    ]),
    
    html.Div([dcc.Input(id='sender_password_input', type='password', placeholder='Enter your password...'),
              html.Button('Send Birthday Wishes', id = 'send_bday_mail', n_clicks = 0, className = 'text-center'), 
              html.Button('Send Work Anniversary Wishes', id = 'send_anni_mail', n_clicks = 0, className = 'text-center')], 
             style = {'display':'flex', 'margin': '15px'}),#, className = 'row')
    
    html.Div(id='current-month-bday-info'), #fro send bday wish button
    
    html.Div(id='current-month-anni-info') #anni mails ke liye 
])
#  ],style={
#         'background': 'linear-gradient(360deg, #FF5733, #FFC300)',  # Replace with your desired gradient colors
#         'height': '100vh',  # Set the height to cover the entire viewport
#         'color': 'white',  # Set the text color to contrast with the background
#         'padding': '20px',  # Add padding for content within the div
#     })


In [4]:
# Function to get current month
def get_current_month():
    now = datetime.datetime.now() #the first datetime here is the name of the module being used:"datetime module", 
    #and the second datetime is the name of the "datetime class" within the "datetime module".".now() method is used to 
    #get the current date and time using the system clock and the variable "now" is a datetime object."""
    
    return now.strftime('%m')
#strftime(string format time) method of the datetime class is called on the object "now" to format the date and time in 
#a specified format. "%m" means the month will be displayed as a zero-padded 2-digit number: january as '01', feb as '02' etc

In [5]:
#callbacks have 2 components: callback decorator(has input and output - both having the component id and the component property)
#and the callback function (having the argument -the component property of the input; and the return value -this return value
#gets assigned to the component property of the output)
#so if multiple inputs are present, multiple arguments are to be taken while for multiple outputs, multiple return statements 
#are to be given

In [6]:
# Function to send an email
def send_email(sender_email, password, recv_email, subject, message):
    
    #password = getpass.getpass(prompt="Enter your password: ") #getpass function of the getpass module.prompts the user to 
    #enter password securely. Does not display the characters as they are entered. However,capturing user input in a prompt 
    #is not directly supported in Dash.. toh password input alag se hi lena padega.

    try:
        # Create a message
        msg = MIMEMultipart()
        msg['From'] = sender_email
        msg['To'] = recv_email
        msg['Subject'] = subject

        msg.attach(MIMEText(message, 'plain'))

        # Connect to the SMTP server of the sender's email provider (here, Gmail)
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.starttls()
        server.login(sender_email, password)

        server.sendmail(sender_email, recv_email, msg.as_string())
        server.quit()

        return "Email sent successfully"
    except Exception as e:
        return f"Error: Unable to send email! - {str(e)}"


In [7]:
#Define callback to display persons with DOB or DOJ in the current month
@app.callback(
    Output('current-month-info', 'children'), 
    Input('upload-data', 'contents') #the children property of the current-month-info component is set as output 
    #which dynamically changes when the input is varied. The contents property of the upload-data component is set as input.
)    
def display_current_month_info(contents):
    if contents is None:
        return ''

    content_type, content_string = contents.split(',')
    decoded = io.StringIO(base64.b64decode(content_string).decode('utf-8'))

    df = pd.read_csv(decoded, encoding='utf-8')
    df['date of birth'] = pd.to_datetime(df['date of birth'])
    df['date of joining'] = pd.to_datetime(df['date of joining'])

    current_month = get_current_month()
    birth_month_filter = df['date of birth'].dt.strftime('%m') == current_month #.dt accessor is used to access the date or time component of a pandas dataframe here,'date of birth' 
    joining_month_filter = df['date of joining'].dt.strftime('%m') == current_month

   
    birth_persons = df[birth_month_filter]
    joining_persons = df[joining_month_filter]

    # Create Bootstrap columns for DOB and DOJ
    birth_person_info_divs = []
    joining_person_info_divs = []

    for _, row in birth_persons.iterrows():
        person_name = row['name']
        person_info_div = html.Div([
            html.H5(f'Details for {person_name}:')
        ])

        for column_name, value in row.items():
            person_info_div.children.append(html.P(f'{column_name}: {value}'))

        birth_person_info_divs.append(person_info_div)
        #send_email(sender_email, password, recv_email, "Happy Birthday!", f"Happy birthday, {person_name}!")
   
    for _, row in joining_persons.iterrows():
        person_name = row['name']
        person_info_div = html.Div([
            html.H5(f'Details for {person_name}:')
        ])

        for column_name, value in row.items():
            person_info_div.children.append(html.P(f'{column_name}: {value}'))

        joining_person_info_divs.append(person_info_div)
        #send_email(sender_email, password, recv_email, "Work Anniversary!", f"Happy work anniversary, {person_name}!")

    # Create a Bootstrap row with two columns for DOB and DOJ
    current_month_info_div = html.Div([
        html.Div(birth_person_info_divs, className='col-md-6'),
        html.Div(joining_person_info_divs, className='col-md-6')
    ], className='row')
        
    return current_month_info_div


In [8]:
@app.callback(
    Output('current-month-bday-info', 'children'), 
    Input('upload-data', 'contents'),
    Input('send_bday_mail', 'n_clicks'),
    State('sender_password_input', 'value'),
)
def send_birthday_wishes(contents, n_clicks, password):
    if n_clicks > 0 and contents is not None:
        content_type, content_string = contents.split(',')
        decoded = io.StringIO(base64.b64decode(content_string).decode('utf-8'))
        sender_email = 'arya.verma.923@gmail.com'
        df = pd.read_csv(decoded, encoding='utf-8')
        df['date of birth'] = pd.to_datetime(df['date of birth'])

        current_month = get_current_month()
        birth_month_filter = df['date of birth'].dt.strftime('%m') == current_month
        recipients = df[birth_month_filter][['name', 'email ID']] 
        
        for _, row in recipients.iterrows():
            recipient_name = row['name']
            recipient_email = row['email ID']
            send_email(sender_email, password, recipient_email, "Happy Birthday!", f"Happy birthday, {recipient_name}!")

In [9]:
@app.callback(
    Output('current-month-anni-info', 'children'), 
    Input('upload-data', 'contents'),
    Input('send_anni_mail', 'n_clicks'),
    State('sender_password_input', 'value'),
)
def send_birthday_wishes(contents, n_clicks, password):
    if n_clicks > 0 and contents is not None:
        content_type, content_string = contents.split(',')
        decoded = io.StringIO(base64.b64decode(content_string).decode('utf-8'))
        sender_email = 'arya.verma.923@gmail.com'
        df = pd.read_csv(decoded, encoding='utf-8')
        df['date of joining'] = pd.to_datetime(df['date of joining'])

        current_month = get_current_month()
        birth_month_filter = df['date of joining'].dt.strftime('%m') == current_month
        recipients = df[birth_month_filter][['name', 'email ID']] 
        
        for _, row in recipients.iterrows():
            recipient_name = row['name']
            recipient_email = row['email ID']
            send_email(sender_email, password, recipient_email, "Happy Work Anniversary!", f"Work Anniversary, {recipient_name}!")

In [10]:
# Define combined callback to handle person info display, name dropdown updates, and manual search
@app.callback(
    [
        Output('output-person-info', 'children'),
        Output('name-dropdown', 'options')
    ],
    [
        Input('upload-data', 'contents'),
        Input('name-dropdown', 'value'),
        Input('month-dropdown', 'value'),
        Input('manual-search-button', 'n_clicks'),
        State('manual-search-input', 'value')
    ]
)
def update_person_info(contents, selected_name, selected_month, n_clicks, manual_search_name):
    if contents is None:
        return '', []

    content_type, content_string = contents.split(',')
    decoded = io.StringIO(base64.b64decode(content_string).decode('utf-8'))

    df = pd.read_csv(decoded, encoding='utf-8')
    df['date of birth'] = pd.to_datetime(df['date of birth'])
    df['date of joining'] = pd.to_datetime(df['date of joining'])

    output_person_info = ''
    name_dropdown_options = []

    if selected_month:
        current_month = selected_month
        birth_month_filter = df['date of birth'].dt.strftime('%m') == current_month
        joining_month_filter = df['date of joining'].dt.strftime('%m') == current_month

        filtered_persons = df[birth_month_filter | joining_month_filter]

        person_info_divs = []

        for _, row in filtered_persons.iterrows():
            person_name = row['name']
            person_info_div = html.Div([
                html.H4(f'Details for {person_name}:')
            ])

            # Iterate over all columns in the row and add them to the div
            for column_name, value in row.items():
                person_info_div.children.append(html.P(f'{column_name}: {value}'))

            person_info_divs.append(person_info_div)
            name_dropdown_options.append({'label': person_name, 'value': person_name})

    if selected_name:
        for person_info_div in person_info_divs:
            if person_info_div.children[0].children == f'Details for {selected_name}:':
                output_person_info = person_info_div
                break

    if n_clicks is not None and n_clicks > 0 and manual_search_name:
        person_info = df[df['name'] == manual_search_name]

        if not person_info.empty:
            person_info_dict = person_info.iloc[0].to_dict()
            output_person_info = html.Div([
                html.H4(f'Details for {manual_search_name}:')
            ])

            # Iterate over all columns in the row and add them to the div
            for column_name, value in person_info_dict.items():
                output_person_info.children.append(html.P(f'{column_name}: {value}'))

            name_dropdown_options.append({'label': manual_search_name, 'value': manual_search_name})

    return output_person_info, name_dropdown_options


In [11]:
# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8060)